# Hi Kagglers !

Original notebook : https://www.kaggle.com/code/seabutterfly95/ensemble-of-public-and-beluga-fullbody

---

Here is a simple ensemble method for submissions.
This idea can be easily referenced from previous competitions

If you think this kernel is good, please upvote the following people who provided the original kernel instead.

*  https://www.kaggle.com/aikhmelnytskyy/happywhale-arcface-baseline-eff7-tpu-768-inference

* https://www.kaggle.com/code/nghiahoangtrung/swin-tranform-submission/notebook#Configuration

* https://www.kaggle.com/code/gtownfoster/effv2-l-backfin-embeddings-ensemble


I added my own result training (best82) on fullbody annotation dataset with beluga only
and was able to achieve significant lb boost. 

I use the pytorch notebook by clemchris to train my model with some adjustment such as fine
tuning the margin 


https://www.kaggle.com/code/clemchris/pytorch-backfin-convnext-arcface

In [ ]:
import csv
import pandas as pd 

sub_files = [
                 '../input/effv2-l-backfin-embeddings-ensemble/submission.csv',
                 '../input/swin-tranform-submission/submission.csv',
                 '../input/happywhale-arcface-baseline-eff7-tpu-768-inference/submission.csv',
                '../input/best82/submission.csv'
]

# Weights of the individual subs
sub_weight = [
                0.768**2.4,
                0.830**2.3,
                0.650**6,
                0.650**1.2,
    
            ]


In [ ]:
Hlabel = 'image' 
Htarget = 'predictions'
npt = 6
place_weights = {}
for i in range(npt):
    place_weights[i] = (1 / (i + 1))

print(place_weights)

lg = len(sub_files)
sub = [None]*lg
for i, file in enumerate( sub_files ):   
    print("Reading {}: w={} - {}". format(i, sub_weight[i], file))
    reader = csv.DictReader(open(file,"r"))
    sub[i] = sorted(reader, key=lambda d: str(d[Hlabel]))

out = open("submission.csv", "w", newline='')
writer = csv.writer(out)
writer.writerow([Hlabel,Htarget])

for p, row in enumerate(sub[0]):
    target_weight = {}
    for s in range(lg):
        row1 = sub[s][p]
        for ind, trgt in enumerate(row1[Htarget].split(' ')):
            target_weight[trgt] = target_weight.get(trgt,0) + (place_weights[ind]*sub_weight[s])
    tops_trgt = sorted(target_weight, key=target_weight.get, reverse=True)[:npt]
    writer.writerow([row1[Hlabel], " ".join(tops_trgt)])
out.close()

<center>
    <h1 style='color: #f13658'> Thanks for reading 👍 <h1>